In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
q1 = pd.read_csv("pop20122013.csv")
populations = q1[q1["Année"] == 2013]
populations = populations.loc[:,['Pays','Élément', 'Valeur']]
populations = populations.rename(columns={"Valeur":"Population"})
populations = populations.rename(columns={"Pays":"country"})
populations['Population'] = ((populations['Population'])*1000).astype(int)
def traitement(dataset):
    dataset = dataset[dataset["country"] != "Chine"]
    return dataset
populations=traitement(populations)
popmond=populations["Population"].sum()
print("La population mondiale est de ",popmond," habitants")
populations.head()

La population mondiale est de  6997326000  habitants


,country,Élément,Population
1,Afghanistan,Population totale,30552000
3,Afrique du Sud,Population totale,52776000
5,Albanie,Population totale,3173000
7,Algérie,Population totale,39208000
9,Allemagne,Population totale,82727000


In [53]:
# Import des données des bilans alimentaires
veg = pd.read_csv("veget.csv")
ani = pd.read_csv("anim.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# On renomme les colonnes de temp
temp.columns = ["xx","xx2","country_code","country",'xx3','element'
    ,'item_code','item','xx4',"year","unit","value",'xx5','xx6'
    ,'origin']

# Transformation de temp en table pivot
data_1 = temp.pivot_table(
    index=["country_code","country","item_code","item","year","origin"],
    columns = ["element"], values=["value"], aggfunc=sum)
# On renomme les colonnes (attention l'ordre peut changer selon vos données !)
data_1.columns = ['Feed', 'Other_uses', 'Food_supply_(kcal/capita/day)',
    'Food_supply_quantity_(kg/capita/yr)','Fat_supply_quantity_(g/capita/day)', 'Protein_supply_quantity_(g/capita/day)', 
    'Domestic_supply_quantity', 'Export_Quantity', 'Import_Quantity',
    'Food', 'Losses', 'Production', 
    'Seed', 'Processing', 'Stock_Variation']    
data_1 = data_1.reset_index()
data_1 = traitement(data_1)
data = data_1[data_1["year"] == 2013]
data = data.fillna(0)
data.head()
#temp.head()

,country_code,country,item_code,item,year,origin,Feed,Other_uses,Food_supply_(kcal/capita/day),Food_supply_quantity_(kg/capita/yr),...,Protein_supply_quantity_(g/capita/day),Domestic_supply_quantity,Export_Quantity,Import_Quantity,Food,Losses,Production,Seed,Processing,Stock_Variation
1,1,Arménie,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,...,30.52,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
3,1,Arménie,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,...,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
5,1,Arménie,2514,Maïs,2013,vegetal,96.0,0.0,0.0,0.03,...,0.01,102.0,0.0,82.0,0.0,7.0,21.0,0.0,0.0,0.0
7,1,Arménie,2515,Seigle,2013,vegetal,1.0,0.0,1.0,0.12,...,0.02,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,1,Arménie,2516,Avoine,2013,vegetal,4.0,0.0,2.0,0.37,...,0.09,6.0,0.0,1.0,1.0,0.0,5.0,0.0,0.0,0.0


    Parmi les documents sur les Bilans alimentaires que vous avez téléchargés, il y a des informations redondantes.En effet, pour un pays donné, certaines de ces informations peuvent se calculer à partir d'autres :
    Production (Production)
    Importations (Import Quantity)
    Exportations (Export Quantity)
    Variation de stock (Stock Variation)
    Disponibilité intérieure (Domestic supply)
    Semences (Seed)
    Pertes (Waste)
    Nourriture (Food) , aussi appelée Disponibilité alimentaire (Food Supply)
    Aliments pour animaux (Feed)
    Traitement (Processing)
    Autres utilisations (Other uses)
    
Question 2 : Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici : ). C'est une équation à 3 termes de type \(a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]\) ) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France. Pour avoir un indice, cliquez sur "Définitions et Standards" sur la page de téléchargement des données.

In [54]:
ble_france = data[data["country"] == "France"]
ble_france = ble_france[ble_france["item"] == "Blé"]
ble_france = ble_france[['Production', 'Import_Quantity', 'Export_Quantity', 'Stock_Variation', 'Domestic_supply_quantity', 'Seed', 'Losses', 'Food', 'Feed', 'Processing', 'Other_uses']]
ble_france

,Production,Import_Quantity,Export_Quantity,Stock_Variation,Domestic_supply_quantity,Seed,Losses,Food,Feed,Processing,Other_uses
8289,38614.0,2055.0,21502.0,1131.0,20298.0,748.0,358.0,6971.0,7822.0,1575.0,2824.0


a=Aliments pour animaux+Semences+Pertes+Traitement+Autres Utilisations+Nourriture 
=b=Production+(Importations_Quantité)+Variation de stock-(Exportations_Quantité) 
=c=Disponibilité intérieure


memo : nourriture (population) = Disponibilité alimentaire en quantité(kg/pers/an) = food = food supply

In [55]:
ble_france_a = ble_france[['Seed', 'Losses', 'Food', 'Feed', 'Processing', 'Other_uses']]
somme_a = ble_france_a.sum(axis=1)
somme_a
#a=Aliments pour animaux+Semences+Pertes+Traitement+Autres Utilisations+Nourriture

8289    20298.0
dtype: float64

In [56]:
ble_france_b = ble_france[['Production', 'Import_Quantity', 'Export_Quantity', 'Stock_Variation']]
somme_b = (ble_france_b.xs(key=8289).Production)+(ble_france_b.xs(key=8289).Import_Quantity)+(ble_france_b.xs(key=8289).Stock_Variation)-(ble_france_b.xs(key=8289).Export_Quantity)
somme_b
#b=Production+(Importations_Quantité)+Variation de stock-(Exportations_Quantité)

20298.0

In [57]:
ble_france_c = ble_france[['Domestic_supply_quantity']]
ble_france_c
#c=Disponibilité intérieure

,Domestic_supply_quantity
8289,20298.0


Question 3 : Calcul (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

In [58]:
q3 = data[['country', 'item', 'Food_supply_(kcal/capita/day)', 'Protein_supply_quantity_(g/capita/day)']]
q3 = pd.merge(q3, populations,on=['country'])
q3['Food_supply_kcal'] = (q3['Food_supply_(kcal/capita/day)']*q3['Population'])*365
q3['Protein_supply_quantity_kg'] = (((q3['Protein_supply_quantity_(g/capita/day)'])/1000)*q3['Population'])*365
q3 = q3[['country', 'item', 'Food_supply_kcal', 'Protein_supply_quantity_kg', 'Population']]
q3.head()

,country,item,Food_supply_kcal,Protein_supply_quantity_kg,Population
0,Arménie,Blé,1.112684e+12,33163184.60,2977000
1,Arménie,Orge,0.000000e+00,0.00,2977000
2,Arménie,Maïs,0.000000e+00,10866.05,2977000
3,Arménie,Seigle,1.086605e+09,21732.10,2977000
4,Arménie,Avoine,2.173210e+09,97794.45,2977000


Question 4 : A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [59]:
q4 = data[['country', 'item', 'Food_supply_quantity_(kg/capita/yr)']]
q4 = pd.merge(q3, q4,on=['country', 'item'])
q4 = q4.dropna()
nonnuls = q4["Food_supply_quantity_(kg/capita/yr)"]!=0.00
q4 = q4[nonnuls]
nonnuls = q4["Protein_supply_quantity_kg"]!=0.000000e+00
q4 = q4[nonnuls]
nonnuls = q4["Food_supply_kcal"]!=0.000000e+00
q4 = q4[nonnuls]
q4['Food_supply_quantity_kg'] = (q4['Food_supply_quantity_(kg/capita/yr)']*q4['Population'])
q4['énergie/poids (Kcal/Kg)'] = (q4['Food_supply_kcal']/q4['Food_supply_quantity_kg'])
q4['Proteine/Kg'] = (q4['Protein_supply_quantity_kg']/q4['Food_supply_quantity_kg'])
q4.sort_values(by = 'Proteine/Kg')

,country,item,Food_supply_kcal,Protein_supply_quantity_kg,Population,Food_supply_quantity_(kg/capita/yr),Food_supply_quantity_kg,énergie/poids (Kcal/Kg),Proteine/Kg
15309,Luxembourg,Vin,1.934500e+10,1.934500e+03,530000,51.41,2.724730e+07,709.978603,0.000071
1979,Canada,Edulcorants Autres,1.219936e+12,1.284143e+05,35182000,15.18,5.340628e+08,2284.255599,0.000240
12595,Espagne,Huile de Tournesol,4.864453e+12,1.712836e+05,46927000,12.02,5.640625e+08,8623.960067,0.000304
15228,Belgique,Graisses Animales Crue,8.673334e+11,4.052960e+04,11104000,10.84,1.203674e+08,7205.719557,0.000337
14568,Ouzbékistan,Huile Graines de Coton,1.932647e+12,1.056091e+05,28934000,7.55,2.184517e+08,8847.019868,0.000483
10108,Nicaragua,Pommes,1.553440e+10,4.438400e+04,6080000,14.71,8.943680e+07,173.691366,0.000496
9468,Namibie,Pommes,1.092774e+10,2.521785e+04,2303000,21.31,4.907693e+07,222.665415,0.000514
8455,Malaisie,Huile de Palme,1.778860e+12,1.084671e+05,29717000,6.89,2.047501e+08,8687.953556,0.000530
10631,Tchéquie (la),Edulcorants Autres,1.523430e+11,3.906230e+04,10702000,6.78,7.255956e+07,2099.557522,0.000538
520,Argentine,Edulcorants Autres,9.984341e+11,1.512779e+05,41446000,6.73,2.789316e+08,3579.494799,0.000542


In [60]:
q4bis = q4.pivot_table(values=["énergie/poids (Kcal/Kg)","Proteine/Kg"], index=["item"])
q4bis = q4bis.reset_index()
moyeuf = q4bis[q4bis["item"] == "Oeufs"]
print(moyeuf)
q4bis.head()
#la moyenne calorique mondiale trouvée 136,4 Kcal/100g est en dessous de de norme qui est de 147 kcal/100g
#https://fr.wikipedia.org/wiki/%C5%92uf_(aliment)#Dur_ou_cru
#on remarquera sur wikipedia que le premier producteur mondial est la chine(Chine, continentale	Oeufs	147.8 Kcal/100g)
#on remarque également que globalement,plus le pays est pauvre plus la valeur calorique de l'oeuf est basse.
#payeuf = q4.loc[(q4["item"]=="Oeufs"),["country","item","énergie/poids (Kcal/Kg)"]]
#payeuf.sort_values(by = 'énergie/poids (Kcal/Kg)')

     item  Proteine/Kg  énergie/poids (Kcal/Kg)
51  Oeufs     0.106961              1364.817772


,item,Proteine/Kg,énergie/poids (Kcal/Kg)
0,Abats Comestible,0.176273,1132.473774
1,"Agrumes, Autres",0.004016,260.147069
2,Aliments pour enfants,0.153291,3826.952046
3,Ananas,0.003056,414.062286
4,Animaux Aquatiques Autre,0.043123,618.224052


In [61]:
cheuf = q4.loc[(q4["item"]=="Oeufs") & (q4["country"]=="Chine, continentale"),["country","item","énergie/poids (Kcal/Kg)"]]
cheuf
### à titre informatif valeure calorique d'un oeuf pour la chine continentale

,country,item,énergie/poids (Kcal/Kg)
2567,"Chine, continentale",Oeufs,1478.678038


En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays). Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.

In [62]:
pourprot = q4.loc[:,['country', 'item', 'Proteine/Kg']]
pourprot['pourcentage_prot'] = round(pourprot['Proteine/Kg']*100,2)
pourprot = pourprot.loc[:,['country', 'item', 'pourcentage_prot']]
pourprot.tail()

,country,item,pourcentage_prot
15594,Soudan,"Beurre, Ghee",1.40
15596,Soudan,Oeufs,10.69
15598,Soudan,Poissons Eau Douce,10.68
15602,Soudan,Riz (Eq Blanchi),8.05
15603,Soudan,Lait - Excl Beurre,3.70


In [63]:
protav = pourprot.loc[(pourprot["item"]=="Avoine")]
#protav = protav.sort_values(by = 'pourcentage_prot')
protav.tail()
#Protéines: 10,70 g / 100g  (et pas 14 g / 100g soit 14% pour flocon d'avoine)
#source https://fr.wikipedia.org/wiki/Avoine_cultiv%C3%A9e

,country,item,pourcentage_prot
14626,Venezuela (République bolivarienne du),Avoine,7.18
14802,Éthiopie,Avoine,11.37
15156,Belgique,Avoine,9.13
15251,Luxembourg,Avoine,8.83
15344,Serbie,Avoine,9.73


Question 5 : Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids et 5 aliments parmi les 20 aliments les plus riches en protéines

In [64]:
q5 = q4.pivot_table(values=["énergie/poids (Kcal/Kg)","Proteine/Kg"], index=["item"])
#q5.head()

In [65]:
prok = q5.loc[:,['Proteine/Kg']]
prok = prok.sort_values(by = 'Proteine/Kg')
prok.tail(5)
#5 aliments parmi les 20 aliments les plus riches en protéines

,Proteine/Kg
item,
Légumineuses Autres,0.221129
Pois,0.223035
Arachides Decortiquees,0.250390
Soja,0.315777
Graines Colza/Moutarde,0.365000


In [66]:
enerpoi = q5.loc[:,['énergie/poids (Kcal/Kg)']]
enerpoi = enerpoi.sort_values(by = 'énergie/poids (Kcal/Kg)')
enerpoi.tail(5)
#5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids

,énergie/poids (Kcal/Kg)
item,
Huile de Palme,8655.326472
Huile de Tournesol,8814.348634
Huil Plantes Oleif Autr,8855.128397
Huile de Germe de Maïs,8859.395213
Huile d'Arachide,9125.000000


Question 6 : Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [67]:
q6 = data[['country', 'item', 'origin', 'Domestic_supply_quantity']]
q6 = q6.dropna()
nonnuls = q6["Domestic_supply_quantity"]!=0.0
q6 = q6[nonnuls]
nonnuls = q6["origin"]=="vegetal"
q6= q6[nonnuls]
q6["Domestic_supply_quantity"]=q6["Domestic_supply_quantity"]*1000000
q6= q6.rename(columns={"Domestic_supply_quantity":"Domestic_supply_quantity_(kg)"})
q6 = pd.merge(q6, q4, on=['country', 'item'])
q6 = q6[['country', 'item', 'Domestic_supply_quantity_(kg)', 'énergie/poids (Kcal/Kg)', 'Proteine/Kg']]
q6['Domestic_supply_quantity_Kcal'] = (q6['Domestic_supply_quantity_(kg)'])*(q6['énergie/poids (Kcal/Kg)'])
q6['Domestic_supply_quantity_kg_Prot'] = (q6['Domestic_supply_quantity_(kg)'])*(q6['Proteine/Kg'])
q6 = q6[['country', 'item', 'Domestic_supply_quantity_Kcal', 'Domestic_supply_quantity_kg_Prot']]
DiMKcal=q6['Domestic_supply_quantity_Kcal'].sum()
DiMProt=q6['Domestic_supply_quantity_kg_Prot'].sum()
print("La disponibilité intérieure mondiale pour les produits végétaux uniquement est de ",DiMKcal," Kcal")

La disponibilité intérieure mondiale pour les produits végétaux uniquement est de  1.0448932196165436e+16  Kcal


Question 7 : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

Pour un homme adulte, l'apport conseillé en énergie est, en moyenne, de 2 400 à 2 600 calories par jour, selon l'activité. Pour une femme adulte, il est de 1 800 à 2 200 calories, soit 2250 calories par jour, en moyenne (https://eurekasante.vidal.fr/nutrition/equilibre-alimentaire-adulte/recommandations-nutritionnelles-adulte.html)

In [68]:
Npcal=((DiMKcal/2250)/365)
PNpcal=round((Npcal/popmond)*100,2)
print("Si toute la disponibilité intérieure mondiale en calories des produits végétaux etait utilisé pour l'alimentation, avec un apport moyen de 2250 Kcal/jour il serait possible d'alimenter une population de ",int(Npcal)," personnes, soit ",PNpcal, " % de la population mondiale.")

Si toute la disponibilité intérieure mondiale en calories des produits végétaux etait utilisé pour l'alimentation, avec un apport moyen de 2250 Kcal/jour il serait possible d'alimenter une population de  12723205109  personnes, soit  181.83  % de la population mondiale.


L’apport nutritionnel conseillé en protéines correspond à des apports compris entre 50 et 60 g de protéines pour des individus de poids moyen (60 à 70 kg).soit une moyenne de 55g par personne.
https://www.cerin.org/actualites/lapport-proteique-en-pratique/

In [69]:
Nppro=int((DiMProt/0.055)/365)
PNppro=round((Nppro/popmond)*100,2)
print("Si toute la disponibilité intérieure mondiale en proteines des produits végétaux etait utilisé pour l'alimentation, avec un apport moyen de 55 g de proteines vegetales par jour, il serait possible d'alimenter une population de ",Nppro," personnes, soit ",PNppro, " % de la population mondiale.")

Si toute la disponibilité intérieure mondiale en proteines des produits végétaux etait utilisé pour l'alimentation, avec un apport moyen de 55 g de proteines vegetales par jour, il serait possible d'alimenter une population de  13829904146  personnes, soit  197.65  % de la population mondiale.


Question 8 : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [70]:
q8=data[['country', 'item', 'Food','Feed','Losses','origin']]
q8 = q8[q8["origin"] == "vegetal"]
q8=q8[['country', 'item', 'Food','Feed','Losses']]
q8=q8.fillna(value={"Food":0})
q8=q8.fillna(value={"Feed":0})
q8=q8.fillna(value={"Losses":0})
q8['Total FFL kg'] = ((q8['Food'])+(q8['Feed'])+(q8['Losses']))*1000000
q8 = pd.merge(q8, q4[['country', 'item', 'énergie/poids (Kcal/Kg)','Proteine/Kg']], on=['country', 'item'])
q8['Total Kcal FFL'] = (q8['Total FFL kg'])*(q8['énergie/poids (Kcal/Kg)'])
q8['Total Proteine FFL'] = (q8['Total FFL kg'])*(q8['Proteine/Kg'])
FFLKcal=q8['Total Kcal FFL'].sum()
FFLProt=q8['Total Proteine FFL'].sum()
print("La disponibilité alimentaire mondiale en produits végétaux concernants la nourriture, la nourriture destinée aux animaux et les pertes est de ",int(FFLKcal)," Kcal.")
print("La disponibilité alimentaire mondiale en produits végétaux concernants la nourriture, la nourriture destinée aux animaux et les pertes est de ",int(FFLProt)," Kg de proteines.")
#q8

La disponibilité alimentaire mondiale en produits végétaux concernants la nourriture, la nourriture destinée aux animaux et les pertes est de  8086439034515635  Kcal.
La disponibilité alimentaire mondiale en produits végétaux concernants la nourriture, la nourriture destinée aux animaux et les pertes est de  202433007083  Kg de proteines.


In [71]:
Npcal=((FFLKcal/2250)/365)
PNpcal=round((Npcal/popmond)*100,2)
print("Avec un apport moyen de 2250 Kcal/jour vegetales, il serait possible d'alimenter en calories, une population de ",int(Npcal)," personnes, soit ",PNpcal, " % de la population mondiale si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture")

Avec un apport moyen de 2250 Kcal/jour vegetales, il serait possible d'alimenter en calories, une population de  9846501107  personnes, soit  140.72  % de la population mondiale si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture


In [72]:
Nppro=int((FFLProt/0.055)/365)
PNppro=round((Nppro/popmond)*100,2)
print("Avec un apport moyen de 55 g de proteines vegetales par jour, il serait possible d'alimenter en proteines, une population de ",Nppro," personnes, soit ",PNppro, " % de la population mondiale si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture")

Avec un apport moyen de 55 g de proteines vegetales par jour, il serait possible d'alimenter en proteines, une population de  10083835969  personnes, soit  144.11  % de la population mondiale si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture


Question 9 : Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [73]:
q9=data[['country', 'item', 'Food_supply_quantity_(kg/capita/yr)']]
q9=q9.dropna()
q9 = pd.merge(q9, populations, on=['country'])
q9['Food_supply_quantity_kg']=q9['Food_supply_quantity_(kg/capita/yr)']*q9['Population']
q9=q9[['country', 'item', 'Food_supply_quantity_kg']]
q9 = pd.merge(q9, q4[['country', 'item', 'énergie/poids (Kcal/Kg)','Proteine/Kg']], on=['country', 'item'])
q9['Total_Kcal'] = (q9['Food_supply_quantity_kg'])*(q9['énergie/poids (Kcal/Kg)'])
q9['Total_Proteine'] = (q9['Food_supply_quantity_kg'])*(q9['Proteine/Kg'])
TKcal=int(q9['Total_Kcal'].sum())
TProt=int(q9['Total_Proteine'].sum())
print("La disponibilité alimentaire mondiale est de ",TKcal," Kcal.")
print("La disponibilité alimentaire mondiale  est de ",TProt," Kg de proteines.")
q9.head()

La disponibilité alimentaire mondiale est de  6246899612170000  Kcal.
La disponibilité alimentaire mondiale  est de  206806608695  Kg de proteines.


,country,item,Food_supply_quantity_kg,énergie/poids (Kcal/Kg),Proteine/Kg,Total_Kcal,Total_Proteine
0,Arménie,Blé,388796200.0,2861.868300,0.085297,1.112684e+12,33163184.60
1,Arménie,Seigle,357240.0,3041.666667,0.060833,1.086605e+09,21732.10
2,Arménie,Avoine,1101490.0,1972.972973,0.088784,2.173210e+09,97794.45
3,Arménie,"Céréales, Autres",5834920.0,2607.142857,0.072628,1.521247e+10,423775.95
4,Arménie,Pommes de Terre,134262700.0,663.636364,0.015701,8.910161e+10,2108013.70


In [74]:
Npcal=int(((TKcal/2250)/365))
PNpcal=round((Npcal/popmond)*100,2)
print("Avec un apport moyen de 2250 Kcal/jour, une population de ",Npcal," personnes, soit ",PNpcal, " % de la population pourraient être nourris avec la disponibilité alimentaire mondiale ")

Avec un apport moyen de 2250 Kcal/jour, une population de  7606574870  personnes, soit  108.71  % de la population pourraient être nourris avec la disponibilité alimentaire mondiale 


In [75]:
Nppro=int(TProt/0.055)/365
PNppro=round((Nppro/popmond)*100,2)
print("Avec un apport moyen de 55 g de proteines par jour, une population de ",Nppro," personnes, soit ",PNppro, " % de la population pourraient être nourris avec la disponibilité alimentaire mondiale")

Avec un apport moyen de 55 g de proteines par jour, une population de  10301699063.260275  personnes, soit  147.22  % de la population pourraient être nourris avec la disponibilité alimentaire mondiale


Question 10 : A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [76]:
q10 = pd.read_csv("q10.csv")
q10=q10[['Zone', 'Valeur']]
q10=q10[q10["Valeur"] != "<0.1"]
q10=q10.dropna()
q10[['Valeur']]=q10[['Valeur']].astype(float)
q10[['Valeur']]=q10[['Valeur']]*1000000
q10[['Valeur']]=q10[['Valeur']].astype(int)
q10=q10.rename(columns={"Zone":"country"})
q10=q10.rename(columns={"Valeur":"Nombre_de_personnes_sous_alimentées"})
q10=traitement(q10)
Tsousalim=q10['Nombre_de_personnes_sous_alimentées'].sum()
PTsousalim=round(((Tsousalim/popmond)*100),2)
print("La sous-nutrition represente ",PTsousalim, " % de la population mondiale.")
q10.head()

La sous-nutrition represente  10.63  % de la population mondiale.


,country,Nombre_de_personnes_sous_alimentées
0,Afghanistan,7900000
1,Afrique du Sud,2600000
2,Albanie,200000
3,Algérie,1700000
6,Angola,8100000


Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

Question 11 : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [77]:
q11=data.loc[:,['country', 'item_code', 'item', 'Feed', 'Food']] #ne marche qu avec loc
def cerea(dataset):
    list_produit = "2511","2520","2514","2517","2513","2805","2515","2518"
    dataset['Cereale']=dataset['item_code'].isin(list_produit)
    dataset = dataset[dataset["Cereale"] == True]
    return dataset
q11=cerea(q11)
Total_CA = int(q11['Feed'].sum())
Total_CAH = int(q11['Feed'].sum() + q11['Food'].sum())
print("la proportion des céréales destinées à l'alimentation animale est de ",Total_CA, " millier de tonnes, sur un total (alimentation humaine et animale) de  ", Total_CAH , "millier de tonnes")

la proportion des céréales destinées à l'alimentation animale est de  857284  millier de tonnes, sur un total (alimentation humaine et animale) de   1882391 millier de tonnes


Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition, pour une année choisie.

Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.

Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)

Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure. le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

Question 12 : Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [78]:
q12 = pd.merge(q10[['country']], data[['country', 'item', 'Export_Quantity']], on=['country'])
q12=q12.dropna()
q12=q12.pivot_table('Export_Quantity',index="item", columns="country",aggfunc=sum)
q12["Total_exp"] = q12.sum(axis=1)
q12=q12.sort_values(by='Total_exp', ascending=False)
q12=q12.loc[:,['Total_exp']].head(15)
q12=q12.reset_index()
q12
#Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.

country,item,Total_exp
0,Huile de Palme,46285.0
1,Maïs,37888.0
2,Manioc,35843.0
3,Riz (Eq Blanchi),32787.0
4,Sucre Eq Brut,25237.0
5,Blé,24841.0
6,"Légumes, Autres",19948.0
7,Bananes,17805.0
8,"Fruits, Autres",14590.0
9,Lait - Excl Beurre,14486.0


In [79]:
q12_1 = data[['country', 'item', 'Import_Quantity']]
q12_1 = pd.merge(q12, q12_1, on=['item'])
q12_1 = q12_1[['item', 'country', 'Import_Quantity']]
q12_1=q12_1.sort_values(by='Import_Quantity', ascending=False).head(200)
q12_1
#Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes
#importations de ces produits (1 importation = une quantité d'un produit donné importée par un 
#pays donné sur l'année choisie)

,item,country,Import_Quantity
1748,Soja,"Chine, continentale",63381.0
371,Manioc,"Chine, continentale",29046.0
247,Maïs,Japon,14403.0
900,Blé,Égypte,10331.0
253,Maïs,République de Corée,8755.0
67,Huile de Palme,Inde,8568.0
1601,Lait - Excl Beurre,Allemagne,8332.0
1577,Lait - Excl Beurre,"Chine, continentale",8207.0
1622,Lait - Excl Beurre,Italie,7695.0
874,Blé,Brésil,7630.0


In [80]:
q12_1=q12_1.pivot_table('Import_Quantity',index="item", columns="country")
q12_1["Total_imp"] = q12_1.sum(axis=1)
q12_1=q12_1.sort_values(by='Total_imp', ascending=False)
q12_1=q12_1.loc[:,['Total_imp']]
q12_1
#Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour 
#chacun des 15 produits.

country,Total_imp
item,
Blé,147576.0
Maïs,102451.0
Soja,93455.0
Lait - Excl Beurre,88418.0
Manioc,34655.0
Sucre Eq Brut,34208.0
Huile de Palme,29747.0
"Légumes, Autres",28177.0
"Oranges, Mandarines",22945.0


In [81]:
q12_2 = data.loc[:,['country', 'item', 'Other_uses', 'Domestic_supply_quantity', 'Food', 'Feed']]
q12_2 = pd.merge(q12, q12_2, on=['item'])
q12_2 = q12_2.fillna(value={'Other_uses':0})
q12_x=q12_2.pivot_table(values=["Other_uses","Domestic_supply_quantity","Food","Feed"], index=["item"],aggfunc="sum")
q12_x['ratio_other/domsup'] = ((q12_x['Other_uses'])/(q12_x['Domestic_supply_quantity']))
q12_x['ratio_Feed/ALLFood'] = ((q12_x['Feed'])/((q12_x['Feed'])+(q12_x['Food'])))
ratio1=q12_x.loc[:,['ratio_other/domsup']].sort_values("ratio_other/domsup", ascending = False).head(3)
ratio2=q12_x.loc[:,['ratio_Feed/ALLFood']].sort_values("ratio_Feed/ALLFood", ascending = False).head(3)
print("Les 3 produits parmis les plus exportés par les pays en sous-nutrition, qui ont le ratio le plus important entre la quantité destinés aux Autres utilisations (Other uses) et la disponibilité intérieure au niveau mondial sont :")
print(ratio1)
print("********************************************")
print("Les 3 produits parmis les plus exportés par les pays en sous-nutrition, qui ont le ratio le plus important entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine) an niveau mondial sont :")
print(ratio2)

Les 3 produits parmis les plus exportés par les pays en sous-nutrition, qui ont le ratio le plus important entre la quantité destinés aux Autres utilisations (Other uses) et la disponibilité intérieure au niveau mondial sont :
                ratio_other/domsup
item                              
Huile de Palme            0.698109
Maïs                      0.198144
Manioc                    0.140211
********************************************
Les 3 produits parmis les plus exportés par les pays en sous-nutrition, qui ont le ratio le plus important entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine) an niveau mondial sont :
                     ratio_Feed/ALLFood
item                                   
Maïs                           0.813520
Soja                           0.621391
Poissons Pelagiques            0.464571


Question 13 : Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [82]:
q13_1 = data.loc[(data["origin"]=="vegetal") & (data["country"]=="États-Unis d'Amérique"),["country","item_code","item","Feed"]]
q13_1=cerea(q13_1)
q13_1 = q13_1.dropna()
Tq13_1=int(q13_1['Feed'].sum())
q13_2 = data.loc[(data["origin"]=="animal") & (data["country"]=="États-Unis d'Amérique"),["country","item_code","item","Production"]]
q13_2 = q13_2.dropna()
Tq13_2=int(q13_2['Production'].sum())
Tq13_1
print("La quantitée de céréales destinées à l'alimentation animale aux états-unis est de ",Tq13_1, " milliers de tonnes pour une production de produits animal de  ",Tq13_2, "milliers de tonnes, et une reduction de 10 % de cette production libérerait", Tq13_1/10 ,"milliers de tonnes de céréale." )

La quantitée de céréales destinées à l'alimentation animale aux états-unis est de  138666  milliers de tonnes pour une production de produits animal de   154474 milliers de tonnes, et une reduction de 10 % de cette production libérerait 13866.6 milliers de tonnes de céréale.


Question 14 : En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [83]:
q14_1 = data.loc[(data["country"]=="Thaïlande") & (data["item"]=="Manioc"),["country","item","Export_Quantity","Production"]]
q14_1['Proportion exportée en %'] = round((q14_1['Export_Quantity'])/(q14_1['Production'])*100,2)
Tq14_1a=float(q14_1['Proportion exportée en %'])
q14_2 = pd.merge(q10.loc[(q10["country"]=="Thaïlande")], populations.loc[(populations["country"]=="Thaïlande")], on = ["country"])
q14_2 = q14_2[["country","Nombre_de_personnes_sous_alimentées","Population"]]
q14_2['% de la population en sous-nutrition'] = round((q14_2['Nombre_de_personnes_sous_alimentées'])/(q14_2['Population'])*100,2)
Tq14_2a=float(q14_2['% de la population en sous-nutrition'])
print("La proportion de maniok exporté de thailande represente ",Tq14_1a, "% de sa production et ",Tq14_2a, "% de la population thailandaise est en sous-nutrition." )

La proportion de maniok exporté de thailande represente  83.41 % de sa production et  8.36 % de la population thailandaise est en sous-nutrition.


Une table appelée population, contenant la population de chaque pays pour chaque année comprise entre 2012 et la dernière année disponible à la date où vous aurez consulté le site de la FAO. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.


Question 15 : Proposez une clé primaire pertinente pour cette table 'code_pays' 'année'

In [84]:
import numpy as np
import pandas as pd
import seaborn as sns
q15 = pd.read_csv("pop20122013.csv")
population = q15.loc[:,['Pays','Code Pays','Année','Valeur']]
population.columns = ["pays","code_pays","année","Population"]
population['Population'] = ((population['Population'])*1000).astype(int)
population = population[population["pays"] != "Chine"]
population = population.rename(columns={"Population":"population"})
population = population.set_index(['code_pays', 'année'])#clé primaire
#population.to_csv("population.csv", index = True)
population.head(10)

pays  population
code_pays année                            
2         2012      Afghanistan    29825000
          2013      Afghanistan    30552000
202       2012   Afrique du Sud    52386000
          2013   Afrique du Sud    52776000
3         2012          Albanie     3162000
          2013          Albanie     3173000
4         2012          Algérie    38482000
          2013          Algérie    39208000
79        2012        Allemagne    82800000
          2013        Allemagne    82727000

Une table appelée dispo_alim contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les informations suivantes:
la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
disponibilité alimentaire en tonnes
disponibilité alimentaire en Kcal/personne/jour
disponibilité alimentaire de protéines en g/personne/jour
disponibilité alimentaire de matières grasses en g/personne/jour
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


Question 16 : Proposez une clé primaire pertinente pour cette table."code_pays" "année" "code_produit"

In [85]:
# Import des données des bilans alimentaires
veg = pd.read_csv("q16veg.csv")
ani = pd.read_csv("q16ani.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

temp = temp[temp["Pays"] != "Chine"]
temp.columns = ['Code Domaine', 'Domaine', 'code_pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code Année', 'année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'origin']
dispo_alim = temp.pivot_table(
    index=["code_pays","Pays","Code Produit","Produit","année","origin"],
    columns = ["Élément"], values=["Valeur"], aggfunc=sum)
del temp
dispo_alim = dispo_alim.reset_index()
dispo_alim = dispo_alim.fillna(0)
dispo_alim = dispo_alim.set_index(["code_pays", "année"])
dispo_alim = pd.merge(dispo_alim, population,on=["code_pays","année"])
dispo_alim.columns = ["pays","code_produit","produit","origin","dispo_alim_kcal_p_j","Disponibilité alimentaire en quantité (kg/personne/an)","dispo_mat_gr","dispo_prot","xxx","Population"]
dispo_alim['dispo_alim_tonnes'] = ((dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)'])*(dispo_alim['Population']))/1000
dispo_alim = dispo_alim.loc[:,['pays','produit','code_produit','origin','dispo_alim_tonnes','dispo_alim_kcal_p_j','dispo_prot','dispo_mat_gr']]
dispo_alim = dispo_alim.reset_index()
dispo_alim = dispo_alim.set_index(["code_pays", "année", "code_produit"])#clé primaire
#dispo_alim.to_csv("dispo_alim.csv", index = True)
dispo_alim.head(5)

C:\Users\Marc\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


pays produit   origin  dispo_alim_tonnes  \
code_pays année code_produit                                                
1         2012  2511          Arménie     Blé  vegetal          376884.86   
                2513          Arménie    Orge  vegetal             504.73   
                2514          Arménie    Maïs  vegetal              89.07   
                2515          Arménie  Seigle  vegetal             445.35   
                2516          Arménie  Avoine  vegetal             950.08   

                              dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
code_pays année code_produit                                                 
1         2012  2511                        995.0       29.69          3.53  
                2513                          1.0        0.03          0.01  
                2514                          0.0        0.01          0.00  
                2515                          1.0        0.03          0.01  
                2516                          2.0        0.08          0.03

Une table appelée equilibre_prod contenant pour chaque pays, pour chaque produit, et pour chaque année comprise entre 2012 et la dernière année disponible, les quantités suivantes :
disponibilité intérieure
aliments pour animaux
semences
pertes
transformés
nourriture
autres utilisations
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.


Question 17 : Proposez une clé primaire pertinente pour cette table."code_pays" "année" "code_produit"

In [86]:
q17 = pd.read_csv("q17.csv")
temp = q17[q17["Pays"] != "Chine"]
temp.columns = ['Code Domaine', 'Domaine', 'code_pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code Année', 'année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole']
equilibre_prod = temp.pivot_table(
    index=["code_pays","Pays","Code Produit","Produit","année"],
    columns = ["Élément"], values=["Valeur"], aggfunc=sum)
del temp
equilibre_prod.columns = ["alim_ani","autres_utilisations","dispo_int","nourriture","pertes","semences","transfo"]
equilibre_prod = equilibre_prod.reset_index()
equilibre_prod = equilibre_prod.fillna(0)
equilibre_prod = equilibre_prod.rename(columns={"Pays":"pays"})
equilibre_prod = equilibre_prod.rename(columns={"Code Produit":"code_produit"})
equilibre_prod = equilibre_prod.rename(columns={"Produit":"produit"})
equilibre_prod = equilibre_prod.set_index(["code_pays", "année", "code_produit"])#clé primaire
#equilibre_prod.to_csv("equilibre_prod.csv", index = True)
equilibre_prod.head()

pays produit  alim_ani  autres_utilisations  \
code_pays année code_produit                                                   
1         2012  2511          Arménie     Blé      92.0                  0.0   
          2013  2511          Arménie     Blé      93.0                  0.0   
          2012  2513          Arménie    Orge     121.0                 16.0   
          2013  2513          Arménie    Orge     137.0                 26.0   
          2012  2514          Arménie    Maïs      53.0                  0.0   

                              dispo_int  nourriture  pertes  semences  transfo  
code_pays année code_produit                                                    
1         2012  2511              532.0       377.0    29.0      23.0     10.0  
          2013  2511              554.0       389.0    32.0      30.0     10.0  
          2012  2513              172.0         1.0    14.0      14.0      7.0  
          2013  2513              198.0         0.0    15.0      14.0      7.0  
          2012  2514               58.0         0.0     5.0       0.0      0.0

Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année comprise entre 2012 et la dernière année disponible. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.


Question 18 : Vous vous en doutez... proposez encore une fois une clé primaire pertinente pour cette table ! 'code_pays' 'année'

In [87]:
q18 = pd.read_csv("q18.csv")
temp = q18[q18["Zone"] != "Chine"]
temp = temp.loc[:,['Zone','Code zone','Année','Valeur']]
temp.columns = ['pays', 'code_pays', 'année', 'nb_personnes']
temp = temp[temp["nb_personnes"] != "<0.1"]
temp = temp.dropna()
temp['nb_personnes'] = (temp['nb_personnes']).astype(float)
temp['nb_personnes'] = ((temp['nb_personnes'])*1000000).astype(int)
temp['année'] = temp['année'].replace("2011-2013", "2012")
temp['année'] = temp['année'].replace("2012-2014", "2013")
temp['année'] = temp['année'].replace("2013-2015", "2014")
temp['année'] = temp['année'].replace("2014-2016", "2015")
temp['année'] = temp['année'].replace("2015-2017", "2016")
temp['année'] = temp['année'].replace("2016-2018", "2017")
sous_nutrition = temp
del temp
sous_nutrition = sous_nutrition.set_index(['code_pays', 'année'])#clé primaire
#sous_nutrition.to_csv("sous_nutrition.csv", index = True)
sous_nutrition.head(15)

pays  nb_personnes
code_pays année                              
2         2012      Afghanistan       7100000
          2013      Afghanistan       7900000
          2014      Afghanistan       8800000
          2015      Afghanistan       9600000
          2016      Afghanistan      10200000
          2017      Afghanistan      10600000
202       2012   Afrique du Sud       2400000
          2013   Afrique du Sud       2600000
          2014   Afrique du Sud       2800000
          2015   Afrique du Sud       3200000
          2016   Afrique du Sud       3400000
          2017   Afrique du Sud       3500000
3         2012          Albanie        200000
          2013          Albanie        200000
          2014          Albanie        200000

Question 19 : Écrivez les requêtes SQL permettant de connaître…

Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

Pour chaque année disponible, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

Le nombre de lignes de la table renvoyée sera donc égal à 10 fois le nombre d'années disponibles.

La quantité totale (en kg) de produits perdus par pays et par année.La table renvoyée contiendra donc une ligne par couple (pays, année).

Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [88]:
import pandas as pd
import numpy as np
import sqlite3 as sql
database="Projet_3.db"
connection = sql.connect(database)
query = '''SELECT * FROM sous_nutrition'''
df = pd.read_sql_query(query,connection)
df.head()
#test

,code_pays,année,pays,nb_personnes
0,2,2012,Afghanistan,7100000
1,2,2013,Afghanistan,7900000
2,2,2014,Afghanistan,8800000
3,2,2015,Afghanistan,9600000
4,2,2016,Afghanistan,10200000


Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

In [89]:
query = '''SELECT pays,SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim 
WHERE année="2013"
GROUP BY pays
ORDER BY(dispo_prot_kg)
DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,pays,dispo_prot_kg
0,Islande,48.56690
1,Chine - RAS de Hong-Kong,47.11055
2,Israël,46.72000
3,Lituanie,45.39140
4,Maldives,44.64680
5,Finlande,42.90940
6,Luxembourg,41.47860
7,Monténégro,40.84350
8,Pays-Bas,40.68290
9,Albanie,40.65005


In [90]:
query = '''SELECT pays,SUM(dispo_alim_kcal_p_j)*365 AS dispo_kcal
FROM dispo_alim 
WHERE année="2013"
GROUP BY pays
ORDER BY(dispo_kcal) DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,pays,dispo_kcal
0,Autriche,1376050.0
1,Belgique,1364005.0
2,Turquie,1353420.0
3,États-Unis d'Amérique,1343930.0
4,Israël,1317650.0
5,Irlande,1314730.0
6,Italie,1305970.0
7,Luxembourg,1292100.0
8,Égypte,1284070.0
9,Allemagne,1278595.0


Pour chaque année disponible, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant. Le nombre de lignes de la table renvoyée sera donc égal à 10 fois le nombre d'années disponibles.

In [91]:
query = '''SELECT  *
FROM    (SELECT pays,année, SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim
WHERE année="2013"
GROUP BY pays,année
ORDER BY dispo_prot_kg
ASC LIMIT 10)
UNION
SELECT  *
FROM    (SELECT pays,année, SUM(dispo_prot)/1000*365 AS dispo_prot_kg
FROM dispo_alim
WHERE année="2012"
GROUP BY pays,année
ORDER BY dispo_prot_kg
ASC LIMIT 10)
ORDER BY année,dispo_prot_kg
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,dispo_prot_kg
0,Libéria,2012,13.89555
1,Guinée-Bissau,2012,15.98700
2,Mozambique,2012,17.12580
3,Haïti,2012,17.23165
4,Madagascar,2012,17.24625
5,Congo,2012,18.06385
6,Zimbabwe,2012,18.51280
7,République centrafricaine,2012,19.27200
8,Sao Tomé-et-Principe,2012,19.35960
9,Ouganda,2012,19.59685


La quantité totale (en kg) de produits perdus par pays et par année. La table renvoyée contiendra donc une ligne par couple (pays, année).

In [92]:
query = '''SELECT pays,année, SUM(pertes)*1000000 AS pertes_kg
FROM equilibre_prod
GROUP BY pays,année
ORDER BY pertes_kg
DESC
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,pertes_kg
0,"Chine, continentale",2013,8.957500e+10
1,"Chine, continentale",2012,8.784900e+10
2,Brésil,2013,7.591400e+10
3,Brésil,2012,7.389700e+10
4,Inde,2013,5.593000e+10
5,Inde,2012,5.164000e+10
6,Nigéria,2013,1.985400e+10
7,Nigéria,2012,1.965000e+10
8,Indonésie,2012,1.336900e+10
9,Indonésie,2013,1.308100e+10


Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [93]:
query = '''SELECT population.pays,population.année,sous_nutrition.nb_personnes,population.population,
CAST(nb_personnes AS float) / CAST(population AS float) AS ratio
FROM population
LEFT JOIN sous_nutrition
ON population.pays = sous_nutrition.pays
AND population.année = sous_nutrition.année
WHERE population.année = '2013'
ORDER BY ratio DESC
LIMIT 10
'''
df = pd.read_sql_query(query,connection)
df

,pays,année,nb_personnes,population,ratio
0,Haïti,2013,5200000,10317000,0.504022
1,Zambie,2013,7000000,14539000,0.481464
2,Zimbabwe,2013,6600000,14150000,0.466431
3,République centrafricaine,2013,2000000,4616000,0.433276
4,République populaire démocratique de Corée,2013,10600000,24895000,0.425788
5,Congo,2013,1800000,4448000,0.404676
6,Tchad,2013,4900000,12825000,0.382066
7,Angola,2013,8100000,21472000,0.377235
8,Libéria,2013,1600000,4294000,0.372613
9,Madagascar,2013,8199999,22925000,0.357688


Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [94]:
query = '''SELECT produit,SUM(autres_utilisations)/SUM(dispo_int) AS ratio
FROM equilibre_prod
WHERE année="2013"
GROUP BY produit
ORDER BY ratio
DESC LIMIT 10'''
df = pd.read_sql_query(query,connection)
df

,produit,ratio
0,"Alcool, non Comestible",1.001288
1,Huil Plantes Oleif Autr,0.754744
2,Huile de Palmistes,0.703957
3,Huile de Palme,0.698109
4,Girofles,0.646154
5,Huile de Colza&Moutarde,0.551290
6,Graisses Animales Crue,0.470219
7,Huiles de Poissons,0.448068
8,Huile de Soja,0.416172
9,Plantes Aquatiques,0.379299


Question 20 : pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

Alcool, non Comestible:
-pharmacie : antiseptique(a préparation de la peau avant injection ou ponction ;
l'antisepsie des petites plaies peu étendues, en badigeon avec une compresse ou en pansement humide), désinfectant (petits objets;instruments médicaux, ciseaux, pinces à épiler, thermomètres..et des surfaces inertes)
-Sous forme d'alcool à brûler il sert à alimenter les réchauds à alcool notamment (utilisation pour les réchaud à fondue). Il est constitué d'un mélange d'alcool éthylique (pour 90 %) et de méthanol (de 5 à 10 %) servant de dénaturant.
-L'alcool ménager est de l'éthanol quasi pur à 95 %, dénaturé pour ne pas être bu, et destiné à l'entretien ménager. Les industriels peuvent le parfumer pour en rendre l'utilisation agréable.
https://medicament.ooreka.fr/astuce/voir/724967/alcool-a-70
https://fr.wikipedia.org/wiki/Alcool_d%C3%A9natur%C3%A9

Huil Plantes Oleif Autr:
Code Produit: 2586
Produit Huil: Plantes Oleif Autr
Description :Composition par défaut: 264 Beurre de karité, 266 Huile de ricin, 276 Huile d'abrasin, 278 Huile de jojoba, 281 Huile de carthame, 297 Huile d'oeillette, 306 Suif végétal, 307 Huile de stillingia, 313 Huile de kapok, 334 Huile de lin, 337 Huile de chènevis, 340 Huile végétale nda, 664 Cacao, beurre, 1241 Margarine liquide, 1242 Margarine et graisses alimentaire compactes, 1273 Huile de ricin hydrogénée, 1274 Huile bouillie, déshydraté, etc., 1275 Huile et graisse végétale hydrogénée
utilisations cosmetiques et alimentaires

Une plante oléifère, ou oléagineuse, est une plante dont l'on extrait de l'huile végétale des fruits ou des graines. Par exemple, le colza, le tournesol et l'olivier sont des plantes oléifères. L'huile végétale récupérée de ces plantes peut être destinée à l'alimentation humaine ou à la production de biocarburant pour moteur diesel, sous forme pure ou transestérifiée (EMHV, EEHV)

Huile de Palmistes:
Les débouchés de l’huile de palmiste sont nombreux : savonnerie et cosmétique, oléochimie.
https://www.cirad.fr/nos-recherches/filieres-tropicales/palmier-a-huile/plante-et-usages

Huile de Palme:
Elle entre dans la fabrication de dérivés à usages industriels : acides gras, savons et cosmétiques, savons industriels, encres, résines, esters méthyliques.
L’huile de palme peut aussi être utilisée comme carburant dans les moteurs diesels, soit à partir de l’huile pure, soit après transformation en esther méthylique, mélangé au gazole.
https://www.cirad.fr/nos-recherches/filieres-tropicales/palmier-a-huile/plante-et-usages


Girofles:
Les propriétés antiseptiques et anesthésiques de ces boutons floraux sont reconnues depuis très longtemps et proposées dans les douleurs dentaires. Il entre dans la composition du khôl, primitivement onguent ophtalmique.
En cuisine, il est présent dans le pain d'épices, les biscuits en mélange avec la cannelle, le pot-au-feu, les marinades, la choucroute et il est indispensable à la plupart des currys. Aux Pays Bas, le clou de girofle entier est utilisé pour parfumer un vieux fromage maigre de Frise à pâte dure appelée Nagelkaas (fromage à clous) ou Kanterkaas (AOP)4,5. En Afrique du Nord, le clou de girofle est utilisé en infusion avec le thé.
Le clou de girofle sert de parfum d'ambiance sous forme d'une version végétale de la pomme d'ambre que l'on fabrique en piquant toute la surface d'une orange de clous de girofle.
95 % de la production mondiale de clous de girofle est utilisé pour la fabrication des kreteks, cigarettes indonésiennes.
https://fr.wikipedia.org/wiki/Giroflier


http://www.fao.org/fileadmin/templates/wsfs/docs/Issues_papers/Issues_papers_FR/Comment_nourrir_le_monde_en_2050.pdf

https://www.planetoscope.com/mortalite/32-nombre-de-deces-dus-a-la-malnutrition-dans-le-monde.html

https://www.lejdd.fr/International/voici-pourquoi-la-faim-dans-le-monde-a-encore-progresse-3910056

https://www.lafaimexpliquee.org/La_faim_expliquee/Faits_et_chiffres.html

https://www.lafaimexpliquee.org/La_faim_expliquee/Faits_et_chiffres_files/Des%20chiffres%20et%20des%20faits%20sur%20la%20malnutrition%20dans%20le%20monde.pdf


    Projet_3_Final.ipynb

    Fichiers nécessaires (fao):
    pop20122013.csv
    veget.csv
    anim.csv
    q10.csv
    q16veg.csv
    q16ani.csv
    q17.csv
    q18.csv

    Fichiers générés:
    population.csv
    equilibre_prod.csv
    sous_nutrition.csv
    dispo_alim.csv
    
    Projet_3.db(généré avec sqlitestudio a partir des 4 fichiers précédement générés)

Livrables 
Voici les livrables attendus, que vous transmettrez dans une archive .ZIP :

Le code en R ou en Python qui vous a permis de calculer chacune des réponses aux questions 1 à 14.
Le code en R, Python et/ou SQL vous ayant permis d'enregistrer les données dans la base de données
Les requêtes SQL vous ayant permis de répondre aux questions 15 à 20. 

In [95]:
import numpy as np
import pandas as pd
import seaborn as sns
q1 = pd.read_csv("pop20122013.csv")
populations = q1[q1["Année"] == 2013]
populations = populations.loc[:,['Pays','Élément', 'Valeur']]
populations = populations.rename(columns={"Valeur":"Population"})
populations = populations.rename(columns={"Pays":"country"})
populations['Population'] = ((populations['Population'])*1000).astype(int)
def traitement(dataset):
    dataset = dataset[dataset["country"] != "Chine"]
    return dataset
populations=traitement(populations)
popmond=populations["Population"].sum()
#print("La population mondiale est de ",popmond," habitants")
#population.head()
# Import des données des bilans alimentaires
veg = pd.read_csv("veget.csv")
ani = pd.read_csv("anim.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# On renomme les colonnes de temp
temp.columns = ["xx","xx2","country_code","country",'xx3','element'
    ,'item_code','item','xx4',"year","unit","value",'xx5','xx6'
    ,'origin']

# Transformation de temp en table pivot
data_1 = temp.pivot_table(
    index=["country_code","country","item_code","item","year","origin"],
    columns = ["element"], values=["value"], aggfunc=sum)
# On renomme les colonnes (attention l'ordre peut changer selon vos données !)
data_1.columns = ['Feed', 'Other_uses', 'Food_supply_(kcal/capita/day)',
    'Food_supply_quantity_(kg/capita/yr)','Fat_supply_quantity_(g/capita/day)', 'Protein_supply_quantity_(g/capita/day)', 
    'Domestic_supply_quantity', 'Export_Quantity', 'Import_Quantity',
    'Food', 'Losses', 'Production', 
    'Seed', 'Processing', 'Stock_Variation']    
data_1 = data_1.reset_index()
data_1 = traitement(data_1)
data = data_1[data_1["year"] == 2013]
data = data.fillna(0)
#data.head()
#temp.head()
##############
##curiositée#
############
q9x = data.loc[:,['country', 'item', 'Domestic_supply_quantity','Food','Feed', 'Other_uses', 'Losses','Seed','Processing']]
q9x['Domestic_supply_quantity_kg'] = q9x['Domestic_supply_quantity']*1000000
q9x['Food_kg'] = q9x['Food']*1000000
q9x['Feed_kg'] = q9x['Feed']*1000000
q9x['Other_uses_kg'] = q9x['Other_uses']*1000000
q9x['Losses_kg'] = q9x['Losses']*1000000
q9x['Seed_kg'] = q9x['Seed']*1000000
q9x['Processing_kg'] = q9x['Processing']*1000000
q9x = q9x[['country','item', 'Domestic_supply_quantity_kg', 'Food_kg', 'Feed_kg','Other_uses_kg','Losses_kg','Seed_kg','Processing_kg']]
q9x = q9x.fillna(0)
q9x = q9x.pivot_table(values=['Domestic_supply_quantity_kg','Food_kg','Feed_kg','Other_uses_kg','Losses_kg','Seed_kg','Processing_kg'],index='item',aggfunc='sum')
q9x['%_alimentation'] = round((q9x['Food_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Feed'] = round((q9x['Feed_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Other_uses'] = round((q9x['Other_uses_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Losses'] = round((q9x['Losses_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Seed'] = round((q9x['Seed_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Processing'] = round((q9x['Processing_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x = q9x.reset_index()
q9x = q9x[['item', 'Domestic_supply_quantity_kg', '%_alimentation', '%_Feed','%_Other_uses','%_Losses','%_Seed','%_Processing']]
q9x['%_Total'] = (q9x['%_alimentation'])+(q9x['%_Feed'])+(q9x['%_Other_uses'])+(q9x['%_Losses'])+(q9x['%_Seed'])+(q9x['%_Processing'])
####################
q9x = q9x.sort_values(by='%_Other_uses', ascending=False)
####################

print(q9x['%_Total'].sum()/q9x.shape[0])

q9x

106.9272448979592


,item,Domestic_supply_quantity_kg,%_alimentation,%_Feed,%_Other_uses,%_Losses,%_Seed,%_Processing,%_Total
2,"Alcool, non Comestible",2.174100e+10,0.00,0.00,100.13,0.00,0.00,0.00,100.13
32,Huil Plantes Oleif Autr,6.271000e+09,27.91,0.00,75.47,0.02,0.00,5.39,108.79
40,Huile de Palmistes,6.570000e+09,29.28,0.00,70.40,0.40,0.00,0.00,100.08
39,Huile de Palme,5.040900e+10,30.18,0.00,69.81,0.59,0.00,0.01,100.59
26,Girofles,1.300000e+08,35.38,0.00,64.62,4.62,0.00,0.00,104.62
37,Huile de Colza&Moutarde,2.341600e+10,41.87,2.99,55.13,0.01,0.00,0.00,100.00
30,Graisses Animales Crue,2.295100e+10,45.28,6.78,47.02,0.19,0.00,1.04,100.31
46,Huiles de Poissons,8.280000e+08,8.09,46.74,44.81,0.00,0.00,0.00,99.64
41,Huile de Soja,4.196100e+10,58.01,0.03,41.62,0.35,0.00,0.00,100.01
68,Plantes Aquatiques,2.355400e+10,60.74,1.32,37.93,0.00,0.00,0.00,99.99


In [96]:
import numpy as np
import pandas as pd
import seaborn as sns
q1 = pd.read_csv("pop20122013.csv")
populations = q1[q1["Année"] == 2013]
populations = populations.loc[:,['Pays','Élément', 'Valeur']]
populations = populations.rename(columns={"Valeur":"Population"})
populations = populations.rename(columns={"Pays":"country"})
populations['Population'] = ((populations['Population'])*1000).astype(int)
def traitement(dataset):
    dataset = dataset[dataset["country"] != "Chine"]
    return dataset
populations=traitement(populations)
popmond=populations["Population"].sum()
#print("La population mondiale est de ",popmond," habitants")
#population.head()
# Import des données des bilans alimentaires
veg = pd.read_csv("veget.csv")
ani = pd.read_csv("anim.csv")

# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# On renomme les colonnes de temp
temp.columns = ["xx","xx2","country_code","country",'xx3','element'
    ,'item_code','item','xx4',"year","unit","value",'xx5','xx6'
    ,'origin']

# Transformation de temp en table pivot
data_1 = temp.pivot_table(
    index=["country_code","country","item_code","item","year","origin"],
    columns = ["element"], values=["value"], aggfunc=sum)
# On renomme les colonnes (attention l'ordre peut changer selon vos données !)
data_1.columns = ['Feed', 'Other_uses', 'Food_supply_(kcal/capita/day)',
    'Food_supply_quantity_(kg/capita/yr)','Fat_supply_quantity_(g/capita/day)', 'Protein_supply_quantity_(g/capita/day)', 
    'Domestic_supply_quantity', 'Export_Quantity', 'Import_Quantity',
    'Food', 'Losses', 'Production', 
    'Seed', 'Processing', 'Stock_Variation']    
data_1 = data_1.reset_index()
data_1 = traitement(data_1)
data = data_1[data_1["year"] == 2013]
data = data.fillna(0)
#data.head()
#temp.head()
##############
##curiositée#
############
q9x = data.loc[:,['country', 'item', 'Domestic_supply_quantity','Food','Feed', 'Other_uses', 'Losses','Seed','Processing']]
q9x['Domestic_supply_quantity_kg'] = q9x['Domestic_supply_quantity']*1000000
q9x['Food_kg'] = q9x['Food']*1000000
q9x['Feed_kg'] = q9x['Feed']*1000000
q9x['Other_uses_kg'] = q9x['Other_uses']*1000000
q9x['Losses_kg'] = q9x['Losses']*1000000
q9x['Seed_kg'] = q9x['Seed']*1000000
q9x['Processing_kg'] = q9x['Processing']*1000000
q9x = q9x[['country','item', 'Domestic_supply_quantity_kg', 'Food_kg', 'Feed_kg','Other_uses_kg','Losses_kg','Seed_kg','Processing_kg']]
q9x = q9x.fillna(0)
###########
#q9x = q9x[q9x["item"] == "Avoine"]
q9x = q9x[q9x["item"] == "Huile de Palme"]
###########
q9x = q9x.pivot_table(values=['Domestic_supply_quantity_kg','Food_kg','Feed_kg','Other_uses_kg','Losses_kg','Seed_kg','Processing_kg'],index='country',aggfunc='sum')
q9x['%_alimentation'] = round((q9x['Food_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Feed'] = round((q9x['Feed_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Other_uses'] = round((q9x['Other_uses_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Losses'] = round((q9x['Losses_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Seed'] = round((q9x['Seed_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x['%_Processing'] = round((q9x['Processing_kg'])/(q9x['Domestic_supply_quantity_kg'])*100,2)
q9x = q9x.reset_index()
q9x = q9x.fillna(0)
q9x = q9x[['country', 'Domestic_supply_quantity_kg', '%_alimentation', '%_Feed','%_Other_uses','%_Losses','%_Seed','%_Processing']]
q9x['%_Total'] = (q9x['%_alimentation'])+(q9x['%_Feed'])+(q9x['%_Other_uses'])+(q9x['%_Losses'])+(q9x['%_Seed'])+(q9x['%_Processing'])
q9x = q9x.sort_values(by='Domestic_supply_quantity_kg', ascending=False)
print(q9x['%_Total'].sum()/q9x.shape[0])

#donnee manquantes erronnees ou inexistantes(nan) division par zero faussent les donnees
q9x

90.32841176470588


,country,Domestic_supply_quantity_kg,%_alimentation,%_Feed,%_Other_uses,%_Losses,%_Seed,%_Processing,%_Total
68,Inde,8.413000e+09,19.39,0.0,80.61,0.00,0.0,0.00,100.00
36,"Chine, continentale",7.009000e+09,36.84,0.0,61.41,1.77,0.0,0.00,100.02
69,Indonésie,3.714000e+09,32.31,0.0,67.69,0.00,0.0,0.00,100.00
107,Nigéria,1.947000e+09,48.79,0.0,48.23,2.98,0.0,0.00,100.00
115,Pakistan,1.940000e+09,68.35,0.0,31.60,0.00,0.0,0.00,99.95
4,Allemagne,1.612000e+09,14.70,0.0,86.79,0.00,0.0,0.19,101.68
75,Italie,1.434000e+09,15.55,0.0,84.87,0.00,0.0,0.00,100.42
149,Thaïlande,1.367000e+09,13.68,0.0,90.64,0.00,0.0,0.00,104.32
167,États-Unis d'Amérique,1.304000e+09,0.00,0.0,98.24,0.00,0.0,0.00,98.24
14,Bangladesh,1.278000e+09,43.04,0.0,56.96,0.00,0.0,0.00,100.00
